In [2]:
import numpy as np
from Bio import SeqIO
from Bio import Seq
import csv
import os
from collections import OrderedDict
import gzip
import matplotlib.pyplot as plt
import re
import pandas as pd
from copy import deepcopy

def load_index (file,read_len = 151, trim = "TCTGGTGGATCTGGAGGTCTCGA", Fedit = "", Redit = "", rev = False):
    
    """ Loads in csv file in format of 1) ID 2) sequence as a sequnce cross check. Output is a refrence dictionary that is of form
    {"Seq": (ID,count)} where ID is uniprot ID and goup assingment and count is defaulted to start at 0 reads mapped"""
    
    index = OrderedDict()
    indexfull = []
    with open(file,'r') as f:
        lines = f.readlines()
        header,seqs = lines[0],lines[1:]
        seqs_clean = [s.strip().split(',') for s in seqs]
        
        # For matching with reverse NGS reads, index is reformated with reverse compliment and trimmed sequences. 
        if rev:
            for r in seqs_clean:
                ID = r[0]
                tempseq = r[1]
                cutoff = tempseq.find(trim) #For matching with reverse NGS reads, index is reformated with reverse compliment data. 
                param = 151 - len(Fedit) - len(Redit)  #adust max index length parameter to account for NGS read length and with forward and reverse trims
                
                if len(tempseq) <= param: #Check if unique index sequence is less than total NGS read length
                    fseq = Seq.Seq(Fedit + tempseq[cutoff:-1])  #Apply trims
                    rseq = fseq.reverse_complement() #Take reverse compliment of fwd index sequence
                
                    index[Redit + str(rseq)] = [ID,0] #Create index entry
                    indexfull.append(str(Redit + Seq.Seq(tempseq).reverse_complement())) #append full index in a list of sequence strings with no read trim
                
                else:  #If read is longer than NGS read length, trim from the given read direction to match read length for exact seq match
                    fseq = Seq.Seq(tempseq[-151 + len(Redit):-1])
                    rseq = fseq.reverse_complement()
                                   
                    index[Redit + str(rseq)] = [ID,0]
                    indexfull.append(Redit + str(Seq.Seq(tempseq).reverse_complement()))
                    
        else:
            Redit2 = str(Seq.Seq(Redit).reverse_complement())
            for r in seqs_clean:
                ID = r[0]
                fseq = r[1][:-trim]
                index[fseq] = [ID,0]
        
    return index,indexfull

def process_fastq (fastq_file,fwd,rev,QCavg = 35, QCbase = 20):

    """Loads in FASTQ file of NGS reads data and identified known reverse-sequence region of interest and trims sequence
    to only unique TMD region. Quality of region is determined from FASTQ data and assesses if data passes QC check"""
    
    f_ind = len(fwd) ## Defines forward trim length
    qc_seq = [] ## Initialize list of accetable sequences at trimmed sequence
    
    try:
        with gzip.open(fastq_file,'rt') as f:
            records = SeqIO.parse(f,'fastq') # Load in fastq files
            for r in records:
                r_ind = r.seq.find(rev) # find reverse handle in sequence, if not there assign -1
                qual = r.letter_annotations['phred_quality'][f_ind:r_ind] # Call in quality scores for trimmed region
                if np.mean(qual) >= QCavg: # Check the average quality is above threshold
                    for q in qual:
                        if q <= QCbase: # Check no outlier base is bad quality in mapped region, if so don't append
                            er = 'False'
                            break
                        else:
                            er = 'True'
                    if er:
                        qc_seq.append(str(r.seq[:r_ind]))
                else:
                    continue ## Or jump to next sequnce if QC condition is not met
    except: 
        with open(fastq_file,'rt') as f:
            records = SeqIO.parse(f,'fastq') # Load in fastq files
            for r in records:
                r_ind = r.seq.find(rev) # find reverse handle in sequence, if not there assign -1
                qual = r.letter_annotations['phred_quality'][f_ind:r_ind] # Call in quality scores for trimmed region
                if np.mean(qual) >= QCavg: # Check the average quality is above threshold
                    for q in qual:
                        if q <= QCbase: # Check no outlier base is bad quality in mapped region, if so don't append
                            er = 'False'
                            break
                        else:
                            er = 'True'
                    if er:
                        if r_ind > 0:
                            qc_seq.append(str(r.seq[:r_ind + len(rev)])) ## Append good sequences to cleaned list
                        else:
                            qc_seq.append(str(r.seq[:r_ind]))
                else:
                    continue ## Or jump to next sequnce if QC condition is not met
    return qc_seq,records
        
def map_seqs (qc_seqs,index,fullindex,fwd):
    
    """Maps QC verified sequences to known sequence index for the given library with index defined in previous 
    function 'load_index'. Only exact sequence matches accepted for read counts."""
    
    # Initialize read mapped categories and arrays
    num_reads = 0
    num_mapped = 0
    num_unmapped = 0
    no_fwd_handle = 0
    mapped,unmapped,nohand = [],[],[]
    
    for s in qc_seqs:
        num_reads += 1 #track total reads processed
        
        f_ind = s.find(fwd) #Find index of fwd handle to identify index-matched region
        
        if f_ind >= 0: #If fwd handle is found, look for sequence in the index and increase given index entry read count
            
            if s in index:
                num_mapped += 1
                mapped.append(s)
                index[s][1] += 1
                
            else:  #If sequence is not matched with propper trims check for the sequence appearence in the full index to assign read count
                for i,k in zip(index.keys(),fullindex):
                    loc = re.search(s,k)
                    if loc:
                        num_mapped += 1
                        index[i][1] += 1
                        mapped.append(s)
                        break
                        
                if not loc: #If no match found, move to next read and count sequence as unmapped
                    num_unmapped += 1
                    unmapped.append(s)
        
        else: #If fwd handle not found, ingore read
            no_fwd_handle += 1
            nohand.append(s)
        
    return index,num_reads,num_mapped,mapped,num_unmapped,unmapped,no_fwd_handle,nohand
    
def final_return(fileout,index1,index2 = {},num_reads = []):
    
    """Writes final read data out to file
    """
    
    if len(num_reads) < 1:
        raise Exception("Please pass the number of reads as a list of [Rep1,Rep2]")
    if len(list(index2.keys())) > 0 and len(num_reads) < 2:
        raise Exception("Please pass number of reads for BOTH replicates in a list")
        
    index_temp = deepcopy(index1)
    for k in index_temp.keys():
        index_temp[k].append(index_temp[k][1]/num_reads[0])
    
    
    output_df = pd.DataFrame.from_dict(index_temp,'index',columns = ['ID1','Read Count Rep 1','Read Norm Rep 1'])
    
    if len(list(index2.keys())) > 0:
        rep_index = deepcopy(index2)
        for k in rep_index.keys():
            rep_index[k].append(rep_index[k][1]/num_reads[1])
    
        rep_df = pd.DataFrame.from_dict(rep_index,'index',columns = ['ID2','Read Count Rep 2','Read Norm Rep 2'])
        rep_df_trim = rep_df.loc[:,'ID':'Read Norm Rep 2']
        
        output_df = output_df.join(rep_df_trim)
    with open(fileout,'w') as f:
        output_df.to_csv(f,index=True,index_label = "Sequence")
    return 

def load_and_run(data_file, index, fullindex, fwd_handle = "TCGAGACCTCCAGATCCACCAGA", rev_handle = "GCCCAGATCCTCTTCTGAGATGAG",fwd_tag = "", rev_tag = ""):
    
    """Compiled function to import fastq files and processes index and map read counts using previously defined funcitons.
    """
    
    # Add in stagger to handles in correct order
    fwd_handle_new = fwd_tag + fwd_handle
    rev_handle_new = rev_handle + str(Seq.Seq(rev_tag).reverse_complement())
    
    # Run QC with new handles
    qc_seq,recs = process_fastq(data_file,fwd_handle_new,rev_handle_new)
    index_updated,read_count,map_count,mapped,unmap_count,unmap,notfound_count,notfound = map_seqs(qc_seq,index, fullindex = fullindex, fwd = fwd_handle_new)
    
    print(f'Mapped % : {map_count/read_count}')
    print(f'Unmapped % : {unmap_count/read_count}')
    print(f'No Handle % : {notfound_count/read_count}')
    
    
    return index_updated,read_count,map_count,mapped,unmap_count,unmap,notfound_count,notfound
